In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import itertools

In [ ]:
# Rosenbrock function
def f(x,n):
    y = 0
    for i in range(1,int(n/2+1)):
        temp = 2*(x[2*i]-x[2*i-1]**2)**2 + (1-x[2*i-1])**2
        y += temp
    return y

In [11]:
# Gradient of Rosenbrock function
def grad_f(x,n):
    g = np.zeros(n)
    for i in range(1,int(n/2+1)):
        g[2*i-1] = -4*(x[2*i]-x[2*i-1]**2)*x[2*i-1] - 2*(1-x[2*i-1])
        g[2*i] = 4*(x[2*i]-x[2*i-1]**2)
    return g


In [ ]:
def Hessian_f(x,n):
    H = np.zeros((n,n))
    for i in range(1,int(n/2+1)):
        H[2*i-1,2*i-1] = -4*(x[2*i]-x[2*i-1]**2) + 8*x[2*i-1]**2 + 2
        H[2*i-1,2*i] = -4*x[2*i-1]
        H[2*i,2*i-1] = -4*x[2*i-1]
        H[2*i,2*i] = 4
    return H

In [ ]:
def m(n,x,f,grad_f,Hessian_f,p):
    f_value = f(x,n)
    g_value = grad_f(x,n)
    B_value = Hessian_f(x,n)

    return f_value + g_value.T@p + 0.5*p.T@B_value@p

In [ ]:
def find_tau1(z,p, d, H, g, delta): # need some fix, just make sure tau gets to the boundary
    """
    Find tau such that p_k = z_j + tau * d_j minimizes m_k(p_k) = 0.5 * p.T @ H @ p + g.T @ p
    subject to ||p|| = delta.
    """
    a = d.T @ H @ d
    b = g.T @ d
    c = np.dot(p, p) - delta ** 2

    if a == 0:
        return -b / (2 * np.dot(d, d)) if np.dot(d, d) != 0 else 0.0

    tau = -b / a
    norm_check = np.linalg.norm(p + tau * d)
    if norm_check > delta:
        tau_adjusted = (delta - np.linalg.norm(p)) / np.linalg.norm(d)
        return tau_adjusted

    return tau
    

In [ ]:
def find_tau2(z, p, d, delta): # same as find_tau1
    """
    Find tau such that ||p + tau * d|| = delta.
    """
    a = np.dot(d, d)
    b = 2 * np.dot(z, d)
    c = np.dot(z, z) - delta ** 2

    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        return 0.0

    sqrt_disc = np.sqrt(discriminant)
    tau1 = (-b + sqrt_disc) / (2 * a)
    tau2 = (-b - sqrt_disc) / (2 * a)

    return max(tau1, tau2) if tau1 >= 0 or tau2 >= 0 else 0.0

In [ ]:
def CG_Steihaug(eps,max_iter,delta,f,grad_f,Hessian_f,x,n):
    p = np.zeros(n)
    z = np.zeros(n)
    r = grad_f(x,n)
    d = -r
    f_value = f(x,n)
    g_value = grad_f(x,n)
    B_value = Hessian_f(x,n)
    if np.linalg.norm(r) < eps:
        p = z
        return p
    
    for j in range(max_iter):
        if d.T @ B_value @ d <= 0:
            tau = find_tau1(z,p,d,B_value,g_value,delta)
            p = z + tau*d
            return p
        
        alpha = r.T @ r / (d.T @ B_value @ d)
        z_new = z + alpha*d
        if np.linalg.norm(z_new) >= delta:
            tau = find_tau2(z,p,d,delta)
            p = z + tau*d
            return p
        
        r_new = r + alpha*B_value@d
        if np.linalg.norm(r_new) < eps:
            p = z_new
            return p
        
        beta = r_new.T @ r_new / (r.T @ r)
        d = -r_new + beta*d
        z = z_new
        r = r_new

    return p

In [ ]:
max_iter = 1000
n_list = [2,4,10,100,1000]

for n in n_list:
    x = -np.ones(n) # initial guess
    x_star = np.ones(n)
    delta = 1
    delta_max = 10
    eta = 0.5
    for k in itertools.count():

        eta = min(0.5,np.linalg.norm(grad_f(x,n)))
        eps = eta*np.linalg.norm(grad_f(x,n))
        p_k = CG_Steihaug(eps,f,grad_f)
        p_0 = np.zeros(n)
        rho = (f(x) - f(x + p_k)) / (m(p_0) - m(p_k))
        if rho < 1/4:
            delta = 1/4 * delta
        else:
            if rho > 3/4 and np.linalg.norm(p_k) == delta:
                delta = min(2*delta,delta_max)
            else:
                delta = delta
        if rho > eta:
            x = x + p_k
        else:
            x = x     
    

0
1
2
3
4
5
6
7
8
9
10
